# ctypes

* [ctypes — A foreign function library for Python — Python 3.10.5 documentation](https://docs.python.org/3/library/ctypes.html)
* [python - Getting data from ctypes array into numpy - Stack Overflow](https://stackoverflow.com/questions/4355524/getting-data-from-ctypes-array-into-numpy)
* [numpy配列とctypes配列の相互変換 - Qiita](https://qiita.com/maiueo/items/493d321f0c9a804b3672)

## allocate arrays as ctypes object

In [37]:
import ctypes

In [38]:
from ctypes import *

In [39]:
x = (c_double*(10**6))()
y = (c_double*(10**6))()
z = (c_double*(10**6))()

In [40]:
x, len(x)

(<__main__.c_double_Array_1000000 at 0x7fbb3917b5c0>, 1000000)

In [4]:
x+y

TypeError: unsupported operand type(s) for +: 'c_double_Array_1000000' and 'c_double_Array_1000000'

In [5]:
3.14*y

TypeError: unsupported operand type(s) for *: 'float' and 'c_double_Array_1000000'

* [python - Using __repr__() on a ctypes subclass? - Stack Overflow](https://stackoverflow.com/questions/10708333/using-repr-on-a-ctypes-subclass)

In [10]:
repr(x)

'<__main__.c_double_Array_1000000 object at 0x7ff934623340>'

In [8]:
type(x)

__main__.c_double_Array_1000000

* "Arrays and pointers"
  - [ctypes — A foreign function library for Python — Python 3.10.5 documentation](https://docs.python.org/3/library/ctypes.html#arrays-and-pointers)
* [python - how to know if type is a class _ctypes.PyCArrayType? - Stack Overflow](https://stackoverflow.com/questions/67499696/how-to-know-if-type-is-a-class-ctypes-pycarraytype)

In [7]:
type(type(x))

_ctypes.PyCArrayType

* [python - What does "mro()" do? - Stack Overflow](https://stackoverflow.com/questions/2010692/what-does-mro-do)

In [10]:
type(x).__mro__

(__main__.c_double_Array_1000000, _ctypes.Array, _ctypes._CData, object)

In [16]:
issubclass(type(x), ctypes.Array)

True

In [18]:
def add(self,other):
    z = (c_double*len(self))()
    for i in range(0,len(self)):
        z[i] = self[i]+other[i]
    return z

In [26]:
ctypes.Array.__add__ = add

TypeError: can't set attributes of built-in/extension type '_ctypes.Array'

* [python - How to set a __str__ method for all ctype Structure classes? - Stack Overflow](https://stackoverflow.com/questions/4642774/how-to-set-a-str-method-for-all-ctype-structure-classes)
* [double - How to add/subtract two ctypes.c_double in Python - Stack Overflow](https://stackoverflow.com/questions/57186789/how-to-add-subtract-two-ctypes-c-double-in-python)

In [10]:
class MyArray():

    def __init__(self):
        self.size = 10**6
        self.data = (ctypes.c_double*self.size)()

    def __getitem__(self,idx):
        return self.data[idx]
    
    def __setitem__(self,idx,val):
        self.data[idx]=val
        
    def __add__(self, other):
        ret = self.__class__()
        ret.data = (ctypes.c_double*self.size)()
        for i in range(0,self.size):
            ret[i] = self.data[i]+other.data[i]
        return ret

In [11]:
x = MyArray()
y = MyArray()

In [48]:
x[3]=1.0
y[3]=2.0

In [13]:
%%time
z = x+y

CPU times: user 229 ms, sys: 3.04 ms, total: 232 ms
Wall time: 231 ms


In [14]:
z[3]

3.0

In [19]:
import gc
gc.get_referrers(x), gc.get_referrers(y), gc.get_referrers(z);

In [21]:
import sys
sys.getrefcount(x), sys.getrefcount(y), sys.getrefcount(z)

(2, 2, 2)

---

In [45]:
import random

In [46]:
%%time
for i in range(0,len(x)):
    x[i] = random.random()
    y[i] = random.random()

CPU times: user 266 ms, sys: 0 ns, total: 266 ms
Wall time: 265 ms


In [47]:
%%time
for i in range(0,len(x)):
    z[i] = x[i] + 3.14*y[i]

CPU times: user 225 ms, sys: 788 µs, total: 226 ms
Wall time: 225 ms


## ctypes to numpy ndarray

* [C-Types Foreign Function Interface (numpy.ctypeslib) — NumPy v1.23 Manual](https://numpy.org/doc/stable/reference/routines.ctypeslib.html)

In [41]:
import numpy as np

In [42]:
pointer(x)

In [43]:
x1 = np.ctypeslib.as_array(x)
y1 = np.ctypeslib.as_array(y)
z1 = np.ctypeslib.as_array(z)

In [44]:
type(x1),type(y1),type(z1)

(numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [46]:
x, pointer(x), x1.data

(<__main__.c_double_Array_1000000 at 0x7fbb3917b5c0>,
 <memory at 0x7fbb39c49f40>)

In [49]:
x[3], x1[3]

(1.0, 1.0)

In [52]:
%%time
z1 = x1+3.14*y1

CPU times: user 2.82 ms, sys: 0 ns, total: 2.82 ms
Wall time: 2.35 ms


# C shared object は cffi のものと共用

In [53]:
lib = CDLL("./_triad.cpython-38-x86_64-linux-gnu.so")

In [54]:
z = (c_double*(10**6))()
z[8]

0.0

In [55]:
%%time
lib.triad(pointer(x), pointer(y), pointer(z), len(z))

CPU times: user 1.11 ms, sys: 1.12 ms, total: 2.22 ms
Wall time: 1.73 ms


0

In [56]:
z[8], x[8]+3.14*y[8]

(2.7540259516691803, 2.7540259516691803)

* [C-Types Foreign Function Interface (numpy.ctypeslib) — NumPy v1.23 Manual](https://numpy.org/doc/stable/reference/routines.ctypeslib.html)

In [57]:
clib = np.ctypeslib.load_library("./_triad.cpython-38-x86_64-linux-gnu.so", "")

In [58]:
clib

<CDLL '/home/manabu/SX-Aurora_TSUBASA/Python/./_triad.cpython-38-x86_64-linux-gnu.so', handle 558e83a58a30 at 0x7f1bcc17ed30>

In [59]:
clib.triad

In [60]:
clib.triad.argtypes=[
    np.ctypeslib.ndpointer(dtype=np.float64,ndim=1,flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.float64,ndim=1,flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.float64,ndim=1,flags='C_CONTIGUOUS'),
    c_int64
]

In [61]:
z1 = np.zeros_like(x1)

In [62]:
%%time
clib.triad(x1,y1,z1,len(z1))

CPU times: user 2.26 ms, sys: 0 ns, total: 2.26 ms
Wall time: 1.82 ms


0

In [63]:
z1[9], x1[9]+3.14*y1[9]

(3.0843638852616704, 3.0843638852616704)

* `asarray`
  - [nlcpy/from_data.py at bdd85ca36b5dfc8ea1a60876656099ff5193d07c · SX-Aurora/nlcpy](https://github.com/SX-Aurora/nlcpy/blob/bdd85ca36b5dfc8ea1a60876656099ff5193d07c/nlcpy/creation/from_data.py#L204)
* `send_object_to_ve()`
  - [nlcpy/core.pyx at e307f70ce354586261330163f94239fb97a50d09 · SX-Aurora/nlcpy](https://github.com/SX-Aurora/nlcpy/blob/e307f70ce354586261330163f94239fb97a50d09/nlcpy/core/core.pyx#L1718)
* ``
  - [nlcpy/vememory.pyx at master · SX-Aurora/nlcpy](https://github.com/SX-Aurora/nlcpy/blob/master/nlcpy/core/vememory.pyx)

In [22]:
import numpy as np

In [23]:
x = np.ones((3,))

In [24]:
x

array([1., 1., 1.])

In [25]:
x.data, x.nbytes

(<memory at 0x7fbc0d103640>, 24)

In [31]:
x.dtype

dtype('float64')

In [34]:
x.dtype.char, x.dtype.alignment, x.dtype.byteorder

('d', 8, '=')

* [Data type objects (dtype) — NumPy v1.23 Manual](https://numpy.org/doc/stable/reference/arrays.dtypes.html)

In [36]:
x.dtype.descr

[('', '<f8')]